In [5]:
"""Import all the relevant libraries."""

from mkd_blockchain import MKDBlockchain
from blockchain_utils import BlockchainUtils
import block
from pubsub import pub
from node import Node
from transaction import Transaction
from sensor_transaction import SensorTransaction
from transaction_pool import TransactionPool
from wallet import Wallet
from Crypto.PublicKey import RSA
import pprint
import random
import time
from mkdtree import kdtree


In [6]:
"""Initialize Nodes with Genesis Block"""

print('START')
MAX_NODES = 9999
num_nodes = 80
forging_interval = 10  # Time interval.  10 units of time
nodes = [] #list of all nodes
blockchain_dimensions = 4
genesis_node_id = int(num_nodes/2)

#itterate through and initialize each node
then = time.time()
for i in range(num_nodes):
    new_node = Node(i+1, 0)
    #new_node.blockchain = mkd_blockchain
    new_node.start_listener('c0')
    nodes.append(new_node)
now = time.time()-then
print(f'block init time @ {num_nodes} nodes: {now}')


genesis_forger = nodes[genesis_node_id-1].wallet.public_key_string()
mkd_blockchain = MKDBlockchain(blockchain_dimensions, genesis_node_id, genesis_forger)


publisher = nodes[genesis_node_id-1]

start_broadcast = time.time()
publisher.publish(mkd_blockchain)
completed_broadcast = time.time() - start_broadcast
print(f'time to complete broadcast to {num_nodes} nodes: {completed_broadcast}')


START


KeyboardInterrupt: 

In [ ]:
# """Attempting to add a second block to the kdtree_blockchain"""
#
# attempt_node_id = 5
# attempt_node = nodes[attempt_node_id-1]
#
# attempt_node.test_forge()
#
# kdtree.visualize(attempt_node.blockchain.blocks)
# print(f'Root subtreeHash attempt: {nodes[4].blockchain.blocks.subtree_hash}')
#
# attempt_node_id = 1
# attempt_node = nodes[attempt_node_id-1]
#
# attempt_node.test_forge()
#
# kdtree.visualize(attempt_node.blockchain.blocks)
# print(f'Root subtreeHash attempt: {nodes[4].blockchain.blocks.subtree_hash}')
#
# attempt_node_id = 4
# attempt_node = nodes[attempt_node_id-1]
#
# attempt_node.test_forge()
#
# kdtree.visualize(attempt_node.blockchain.blocks)
# print(f'Root subtreeHash attempt: {nodes[4].blockchain.blocks.subtree_hash}')
#
#
# attempt_node_id = 6
# attempt_node = nodes[attempt_node_id-1]
#
# attempt_node.test_forge()
#
# kdtree.visualize(attempt_node.blockchain.blocks)
# print(f'Root subtreeHash attempt: {nodes[4].blockchain.blocks.subtree_hash}')
#
# attempt_node_id = 5
# attempt_node = nodes[attempt_node_id-1]
#
# attempt_node.test_forge()
#
# kdtree.visualize(attempt_node.blockchain.blocks)
#
# print(f'Root subtreeHash attempt: {nodes[4].blockchain.blocks.subtree_hash}')
#
# test_block = nodes[5].blockchain.blocks.left.data
# print(test_block)
# print(f'Parent Hash: {BlockchainUtils.hash(test_block.to_json()).hexdigest()}')
# child_block = nodes[5].blockchain.blocks.left.right.data
# print(child_block)

In [7]:
# TODO: Need to generate random transactions and need to start implementing test data.


{0: [5], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: [], 12: [], 13: [], 14: [], 15: [], 16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: [], 24: [], 25: [], 26: [], 27: [], 28: [], 29: [], 30: [], 31: [], 32: [], 33: [], 34: [], 35: [], 36: [], 37: [], 38: [], 39: [], 40: [], 41: [], 42: [], 43: [], 44: [], 45: [], 46: [], 47: [], 48: [], 49: [], 50: [], 51: [], 52: [], 53: [], 54: [], 55: [], 56: [], 57: [], 58: [], 59: [], 60: [], 61: [], 62: [], 63: [], 64: [], 65: [], 66: [], 67: [], 68: [], 69: [], 70: [], 71: [], 72: [], 73: [], 74: [], 75: [], 76: [], 77: [], 78: [], 79: []}


In [ ]:
#Opens the data set file and generates nodes based on the provided data.
#file is comma delimited in the following format: time-nodeID-xcoor-ycoor-miner-region-prevregion
then = time.time()
with open('node_data.txt') as f:
    lines = f.readlines() # list containing lines of file
    i = 0


    nodes_transacting = []
    # DICT: key = prev cluster id value = [nodes]
    # TODO: Ensure we traverse all nodes in list because they may have moved to different clusters
    nodes_to_merge = {k: [] for k in range(num_nodes)}

    for line in lines:
        j = i%num_nodes  # j will represent the increments of nodes
        parts = line.split(',')

        moving_node = nodes[j]  # choosing specific node to manipulate

        old_cluster_id = int(moving_node.cluster_id)
        new_cluster_id = int(parts[5])
        old_topic = 'c'+str(old_cluster_id).strip()
        new_topic = 'c' + str(new_cluster_id).strip()

        # gives nodes their coordinates and moves them into their new cluster
        moving_node.coords = [int(parts[2]), int(parts[3])]

        # if node is moving into a new cluster (excluding the genesis cluster) /
        # then we need to record to merge and clear it's transaction pool
        if old_topic != new_topic:
            moving_node.cluster_id = new_cluster_id
            moving_node.move_listener(old_topic, new_topic)
            if old_topic != 'c0':
                nodes_to_merge[old_cluster_id].append(moving_node)

                # clears only if not last node to leave cluster
                moving_node.transaction_pool  = TransactionPool()

        #  generates list of nodes that need to publish a transaction to their cluster
        if int(parts[4]) == 1:
            nodes_transacting.append(moving_node)






        #  Once it reaches the end of each set of nodes from the data file (80 for this first test file)
        if j == (num_nodes-1):
            #  Now it needs to merge first then send transactions


        for node in nodes_transacting:
            transaction = SensorTransaction(node.wallet.public_key_string(), random.randint(0,1000))
            node.publish(transaction)

        # if time to forge then forge and broadcast, needs to scan and perform all clusters
            if int(parts[1])%forging_interval == 0:
                # if transaction pool not empty then forge and broadcast
                # Need to select forger then use mkd_forge (change return to broadcast)







        i += 1


now = time.time() - then
print(f'Complete movement simulation @ {num_nodes} nodes: {now}')